In [1]:
import scipy as sp
import numpy as np
from numbers import Number

In [ ]:
from numbers import Number

def shipping_analysis(A, b, M, kra=True, eq=True, flag=True):
    if not (isinstance(A, np.ndarray) and len(A.shape) == 2
            and isinstance(b, np.ndarray) and len(b.shape) == 1
            and isinstance(M, Number)):
        raise TypeError("A has to be a 2D numpy array, b a 1D numpy array"
                        "and M a number.")
    
    if kra:
        A_w = A
        b_w = b
        c1 = np.array([1, 1, 0, 0, 0, 0])
        c2 = np.array([-1, 0, 1, 0, 0, 1])
        c3 = np.array([0, -1, 0, 1, 1, 0])
    else:
        A_w = A[:-1, :-1]
        b_w = b[:-1]
        c1 = np.array([1, 1, 0, 0, 0])
        c2 = np.array([-1, 0, 1, 0, 0])
        c3 = np.array([0, -1, 0, 1, 1])

    x0 = np.ones_like(b_w)
    bounds = [(0, None) for _ in range(len(x0))]
    def constraint_1(x):
        return c1 @ x - M
    def constraint_2(x):
        return c2 @ x
    def constraint_3(x):
        return c3 @ x
    if eq:
        def cost_function(x):
            if not isinstance(x, np.ndarray):
                raise TypeError
            return 1/2 * (x.T @ A_w @ x) + b_w.T @ x
    else:
        def cost_function(x):
            if not isinstance(x, np.ndarray):
                raise TypeError
            return 1/M*(x.T @ A_w @ x + b_w.T @ x)
    constraints = [
            {"type": "eq", "fun": constraint_1},
            {"type": "eq", "fun": constraint_2},
            {"type": "eq", "fun": constraint_3},
        ]
    result = sp.optimize.minimize(cost_function, x0, constraints=constraints, 
                                  bounds=bounds, tol=1e-8)
    if flag:
        if eq:
            text = "Equilibrium Analysis"
        else:
            text = "Social Optimum Analysis"
        if kra:
            text += " (With Kra Canal)"
        else:
            text += " (Without Kra Canal)"
        print(text)
        if result.success:
            print("Optimal solution:", result.x)
            print("Objective value:", result.fun)
        else:
            print("Optimization failed:", result.message)
    return result.x
A = np.diag([0.1, 2, 3, 4, 5, 6])
b = np.array([0.1, 3, 4, 5, 6, 7])
M = 100
x = shipping_analysis(A, b, M, kra=False, eq=False, flag=True)

Social Optimum Analysis (Without Kra Canal)
Optimal solution: [57.95987387 42.04012613 57.95987387 23.41120169 18.62892445]
Objective value: 184.68827048616757


In [5]:
def route_check(x):
    if len(x) == 6:
        A_w = A
        b_w = b
        r = np.array([[1, 0, 0, 0, 0, 1],
                      [1, 0, 1, 0, 0, 0],
                      [0, 1, 0, 1, 0, 0],
                      [0, 1, 0, 0, 1, 0]])
    elif len(x) == 5:
        A_w = A[:-1, :-1]
        b_w = b[:-1]
        r = np.array([[1, 0, 1, 0, 0],
                      [0, 1, 0, 1, 0],
                      [0, 1, 0, 0, 1]])
    else:
        raise ValueError('x needs to be 5 or 6 dimensional.')
    return r @ (A_w @ x + b_w)

def social_optimum(x):
    if len(x) == 6:
        A_w = A
        b_w = b
    elif len(x) == 5:
        A_w = A[:-1, :-1]
        b_w = b[:-1]
    return (x.T @ A_w @ x + b_w .T @ x) / M
social_optimum(x)

np.float64(184.68827048616757)